In [5]:
import findspark
import pandas
findspark.init()
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName('Sentiment').master('local[*]') \
                    .config("spark.eventLog.enabled", "true") \
                    .config("spark.eventLog.dir", "/tmp/spark-history") \
                    .config("spark.sql.files.maxPartitionBytes","268435456") \
                    .config("spark.executor.memory","5g") \
                    .config("spark.sql.shuffle.partitions","40") \
                    .config("spark.ui.ProxyBase","") \
                    .getOrCreate()


In [4]:
spark.stop()

In [14]:
print(spark.sparkContext.getConf().getAll())

[('spark.sql.shuffle.partitions', '40'), ('spark.eventLog.enabled', 'true'), ('spark.driver.port', '38575'), ('spark.app.name', 'Sentiment'), ('spark.ui.ProxyBase', ''), ('spark.executor.id', 'driver'), ('spark.executor.memory', '5g'), ('spark.driver.host', '192.168.1.13'), ('spark.sql.files.maxPartitionBytes', '268435456'), ('spark.history.fs.logDirectory', 'file:///tmp/spark-history'), ('spark.eventLog.dir', '/tmp/spark-history'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.app.id', 'local-1605375759066'), ('spark.ui.showConsoleProgress', 'true')]


In [10]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, TimestampType, FloatType
from pyspark.sql.functions import to_date, date_format, isnull, udf, col,lit, to_timestamp, sum, count, avg
# from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import os
import re

In [16]:
sia = SIA()

# def measure_sentiment(text):
#     txtblob = TextBlob(text)
#     return txtblob.sentiment.polarity

# def measure_subjectivity(text):
#     txtblob = TextBlob(text)
#     return txtblob.sentiment.subjectivity

def measure_sia(text):
    return sia.polarity_scores(text)['compound']

def clean_up(text):
    # Remove mentions
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    # Remove hashtags
    text = re.sub(r'#', '', text)
    # Remove retweets:
    text = re.sub(r'RT : ', '', text)
    # Remove urls
    text = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', text)
    return text

# measure_sentiment_udf = udf(lambda z: measure_sentiment(z), FloatType())
# spark.udf.register("measure_sentiment_udf", measure_sentiment_udf)

# measure_subjectivity_udf = udf(lambda z: measure_subjectivity(z), FloatType())
# spark.udf.register("measure_subjectivity_udf", measure_subjectivity_udf)

clean_up_udf = udf(lambda z: clean_up(z), StringType())
spark.udf.register("clean_up_udf", clean_up_udf)

measure_sia_udf = udf(lambda z: measure_sia(z), FloatType())
spark.udf.register("measure_sia_udf", measure_sia_udf)

<function __main__.<lambda>(z)>

In [17]:
original = spark.read.csv(path='./tweets.csv', inferSchema=True, header=True, sep=';')

df = original.select('timestamp', 'text','replies','likes','retweets').dropna() \
            .withColumn('datetime',date_format('timestamp','yyyy-MM-dd HH:mm:ss')).dropna().drop('timestamp')

df = df.filter( (df.datetime > '2016-01-01 00:00:00') & (df.datetime < '2019-11-23 00:00:00'))

df = df.withColumn('datetime', date_format('datetime','yyyy-MM-dd HH')) \
    .withColumn('datetime', date_format('datetime','yyyy-MM-dd HH:mm')) \
    .withColumn('datetime', to_timestamp(col('datetime'))) \
    .withColumn('replies', col('replies').cast(IntegerType())) \
    .withColumn('likes', col('likes').cast(IntegerType())) \
    .withColumn('retweets', col('retweets').cast(IntegerType()))

df = df.withColumn('text', clean_up_udf(col('text'))) \
      .filter(col('text') != '') \
      .withColumn('score', measure_sia_udf(col('text'))) \
      .filter(col('score') != 0) \
      .drop('text')

df.write.parquet(path=os.path.join(os.getcwd(), 'df'), mode='overwrite')

Py4JJavaError: An error occurred while calling o350.csv.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.GatewayConnection.run(GatewayConnection.java:238)
java.lang.Thread.run(Thread.java:748)

The c

In [ ]:
df = spark.read.load(path=os.path.join(os.getcwd(),'df/*.parquet'),
                     format="parquet", header=True, inferSchema=True)
df.printSchema()
df.show(50,False)

In [ ]:
df = spark.read.load(path=os.path.join(os.getcwd(),'df/*.parquet'),
                     format="parquet", header=True, inferSchema=True)


# df.printSchema()
df = df.groupBy('datetime').agg(avg('score').alias('score'), \
                                avg('replies').alias('replies'), \
                                avg('likes').alias('likes'), \
                                avg('retweets').alias('retweets')) \
                            .sort(['datetime'], ascending=True)


df.write.parquet(path=os.path.join(os.getcwd(), 'df_grouped'), mode='overwrite')

In [ ]:
##########################################################################################################

In [11]:
df = spark.read.load(path=os.path.join(os.getcwd(),'df/*.parquet'),
                     format="parquet", header=True, inferSchema=True)

df = df.groupBy('datetime').agg(count('score').alias('score'), \
                                sum('replies').alias('replies'), \
                                sum('likes').alias('likes'), \
                                sum('retweets').alias('retweets')) \
                            .sort(['datetime'], ascending=True)
df.coalesce(1).write.format("csv").option("header", "true") \
                            .option('timestampFormat', 'yyyy-MM-dd HH:mm').save("temp_sent")

In [ ]:
df = spark.read.load(path=os.path.join(os.getcwd(),'df_grouped/*.parquet'),
                     format="parquet", header=True, inferSchema=True)

df.coalesce(1).write.format("csv").option("header", "true") \
                            .option('timestampFormat', 'yyyy-MM-dd HH:mm').save("sentiment")



